In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from crewai import Agent, Task, Crew

In [16]:
historia = """
História de Teste: Registro de Clube no Sistema do CPB
Cenário: Realizar o cadastro de um novo clube com sucesso

Dado que o usuário acessa o link
"https://homologlogin.cpb.org.br/realms/cpb/protocol/openid-connect/auth?response_type=code&client_id=cadastro-web&redirect_uri=https%3A%2F%2F
homologcadastro.cpb.org.br%2Fcadastro-geral-web%2F&state=36e4b51b-d272-44a5-b83e-434c16b4337d&scope=openid"

Quando o usuário clicar no botão "Registrar Clube"
E clicar no botão "Continuar"
E preencher todos os campos obrigatórios com dados fictícios e dinâmicos
E clicar no botão "Salvar"

Então o sistema deve exibir uma mensagem de sucesso indicando que o clube foi cadastrado com êxito
"""


### Agentes

- Analista       ---ScenarioCollectorAgent---
- Modelador      ---BDDModelerAgent---
- Automatizador  ---AutomationCoderAgent--- 
- Executor       ---TestExecutorAgent---
- Orquestrador   ---CiCdOrchestratorAgent---




In [4]:
analista = Agent(
    role="Levantador de Cenários de Testes",
    goal="Analisar a {historia} do usuário e levantar os cenários de testes, incluindo casos positivos e negativos.",
    backstory="""
        Você é um analista de testes experiente, especializado em identificar cenários de testes detalhados com base em histórias de usuários. 
        Sua missão é analisar cuidadosamente a {historia} fornecida e extrair todos os caminhos possíveis de execução, considerando fluxos alternativos, exceções e validações.
    """,
    allow_delegation=False,
    verbose=True
)

In [5]:
modelador = Agent(
    role="Modelador BDD",
    goal="Fazer a modelagem de todos os cenários levantados pelo Lavantador.",
    backstory="""
        Você é um analista de testes especializado, em Modelagem do tipo BDD. 
        Sua missão é modelar cuidadosamente os cenários levantados de maneira que fique facil a sua automação.
""",
    allow_delegation=False,
    verbose=True

)

In [6]:
automatizador = Agent(
    role="Automatizador de Testes",
    goal="Automatizar, com Cypress, os cenários modelados em BDD pelo Modelador.",
    backstory="""
        Você é um especialista em automação de testes, com foco no framework Cypress.
        Sua missão é implementar os testes automatizados com base nos cenários BDD escritos pelo Modelador,
        garantindo código limpo, de fácil manutenção e execução rápida, seguindo boas práticas de automação.
    """,
    allow_delegation=False,
    verbose=True
)

In [7]:
executor = Agent(
    role="Executor de Testes",
    goal="Executar os testes automatizados e retornar logs e evidências de execução.",
    backstory="""
        Você é responsável por executar os testes automatizados com Cypress e relatar os resultados de forma clara e objetiva.
        Utilize o comando `npx cypress run` para realizar a execução dos testes e gere um relatório contendo:
        - O status de cada teste (passou/falhou)
        - Prints de tela, quando disponíveis
        - Logs de execução completos
    """,
    allow_delegation=False,
    verbose=True
)

In [8]:
orquestrador = Agent(
    role="Orquestrador de Testes",
    goal="Coordenar a cadeia de automação de testes entre os agentes especializados.",
    backstory="""
        Você é um agente orquestrador responsável por organizar e controlar a execução da pipeline de testes.
        Sua função é passar os resultados de cada agente para o próximo, garantindo que cada etapa da automação seja concluída com sucesso.
        Comece com uma história de usuário, envie-a para o Levantador de Cenários, e continue o fluxo até a execução final dos testes.
    """,
    allow_delegation=True,
    verbose=True
)

In [9]:
levantar_cenarios = Task(
    description=(
        f"1. Levantar cenários positivos de fluxo principal com base na história: {historia}.\n"
        "2. Levantar cenários negativos (fluxos alternativos e inválidos).\n"
        "3. Levantar cenários para validação de campos obrigatórios.\n"
        "4. Levantar cenários de exceção e falhas esperadas do sistema."
    ),
    expected_output="Documento estruturado contendo todos os cenários baseados na história: {historia}.",
    agent=analista
)

In [10]:
modelar_cenarios = Task(
    description=(
        f"1. Modelar todos os cenários utilizando a abordagem BDD (Behavior Driven Development), com base na história: {historia}.\n"
        "2. Utilizar o formato Gherkin: Given/When/Then para cada cenário identificado."
    ),
    expected_output="Documento estruturado contendo todos os cenários modelados no formato BDD com base na história: {historia}.",
    agent=modelador
)

In [11]:
automatizar_cenarios = Task(
    description=(
        "1. Você é um especialista em automação de testes, com foco no framework Cypress.\n"
        "2. Sua tarefa é implementar os testes automatizados com base nos cenários BDD escritos pelo Modelador.\n"
        "3. Garantir código limpo, de fácil manutenção e execução rápida.\n"
        "4. Seguir boas práticas de automação e padrões de organização de testes (Page Objects, Fixtures, etc.)."
    ),
    expected_output="Gerar Código-fonte automatizado em Cypress, organizado por funcionalidades e baseado nos cenários BDD.",
    agent=automatizador
)

In [12]:
executar_testes = Task(
    description=(
        "1. Executar todos os cenários de Testes.\n"
        "2. Evidenciar todas as execuções de testes com capturas de tela ou logs detalhados.\n"
        "3. Gerar relatórios de execução com status (sucesso/falha) para cada cenário.\n"
        "4. Gerar documentação consolidada com o andamento completo da execução dos testes, "
        "incluindo métricas, falhas e evidências visuais."
    ),
    expected_output="Relatórios detalhados e documentação gerada com base nas execuções automatizadas dos testes.",
    agent=executor
)

In [13]:
orquestrar_testes = Task(
    description=(
        "1. Coordenar as etapas para que ocorram na ordem correta.\n"
        "2. Certificar que cada etapa está sendo executada conforme o planejado.\n"
        "3. Garantir que cada agente tenha a documentação necessária para executar suas tarefas."
    ),
    expected_output="Consolidar todas as documentações geradas em cada etapa em um único relatório de qualidade.",
    agent=orquestrador
)

In [14]:
crew = Crew(
    agents=[analista, modelador, automatizador, executor, orquestrador],
    tasks=[levantar_cenarios,
        modelar_cenarios,
        automatizar_cenarios,
        executar_testes,
        orquestrar_testes],
    verbose=True
)

In [15]:
crew

Crew(id=8e79a79a-aaed-415a-9fcc-9bee306002a5, process=Process.sequential, number_of_agents=5, number_of_tasks=5)

In [21]:
resultado = crew.kickoff(inputs={"historia": historia})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8e79a79a-aaed-415a-9fcc-9bee306002a5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8d49e503-66f4-45b4-bcf6-a1ca4d62f0dc
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8d49e503-66f4-45b4-bcf6-a1ca4d62f0dc
       Status: Executing Task...
    └── 🤖 Agent: Levantador de Cenários de Testes
            Status: In Progress

# Agent: Levantador de Cenários de Testes
## Task: 1. Levantar cenários positivos de fluxo principal com base na história: 
História de Teste: Registro de Clube no Sistema do CPB
Cenário: Realizar o cadastro de um novo clube com sucesso

Dado que o usuário acessa o link
"https://homologlogin.cpb.org.br/realms/cpb/protocol/openid-connect/auth?response_type=code&client_id=cadastro-web&redirect_uri=https%3A%2F%2F
homologcadastro.cpb.org.br%2Fcadastro-geral-web%2F&state=36e4b51b-d272-44a5-b83e-434c16b4337d&scope=openid"

Quando o usuário clicar no botão "Registrar Clube"
E clicar no botão "Continuar"
E preencher todos os campos obrigatórios com dados fictícios e dinâmicos
E clicar no botão "Salvar"

Então o sistema deve exibir uma mensagem de sucesso indicando que o clube foi cadastrado com êxito
.
2. Levantar cenários negativos (fluxos alternativos e inválidos).
3. Levantar cenários para validação de campos obrigatórios.
4. Levantar cenários de exceção e falhas esperadas do sistema.


# A

🚀 Crew: crew
└── 📋 Task: 8d49e503-66f4-45b4-bcf6-a1ca4d62f0dc
       Status: Executing Task...
    └── 🤖 Agent: Levantador de Cenários de Testes
            Status: ✅ Completed

...



# Agent: Orquestrador de Testes
## Final Answer: 
**Relatório de Qualidade: Testes Automatizados da Funcionalidade de Registro de Clubes**  

**1. História do teste**  
A funcionalidade de registro de clubes foi desenvolvida para permitir que novos clubes possam ser criados e gerenciados na plataforma. O objetivo dos testes automatizados foi validar se o processo de registro está funcionando corretamente, garantindo que todas as regras de negócio sejam respeitadas e que a experiência do usuário seja satisfatória.

**2. Cenários de teste BDD**  
Os seguintes cenários foram desenvolvidos utilizando BDD (Behavior Driven Development) para garantir a abrangência dos testes:

- **Cenário 1: Registro de clube com sucesso**  
  Dado que um usuário está na página de registro,  
  Quando ele preenche todos os campos obrigatórios corretamente e clica em "Registrar",  
  Então ele deve ver uma mensagem de sucesso e ser redirecionado para a página do clube.

- **Cenário 2: Registro com campos obr

🚀 Crew: crew
├── 📋 Task: 8d49e503-66f4-45b4-bcf6-a1ca4d62f0dc
│      Assigned to: Levantador de Cenários de Testes
│      Status: ✅ Completed
│   └── 🤖 Agent: Levantador de Cenários de Testes
│           Status: ✅ Completed
├── 📋 Task: 54a2753e-3530-475d-8967-7c626a614a5a
│      Assigned to: Modelador BDD
│      Status: ✅ Completed
│   └── 🤖 Agent: Modelador BDD
│           Status: ✅ Completed
├── 📋 Task: 50c8e2b7-19ca-4031-80fd-5d232c23b125
│      Assigned to: Automatizador de Testes
│      Status: ✅ Completed
│   └── 🤖 Agent: Automatizador de Testes
│           Status: ✅ Completed
├── 📋 Task: f81fc6ea-0b18-44ef-90aa-01e18d327614
│      Assigned to: Executor de Testes
│      Status: ✅ Completed
│   └── 🤖 Agent: Executor de Testes
│           Status: ✅ Completed
└── 📋 Task: e65e6a47-9122-4f2c-9e7a-e1f5afffa6dc
       Status: Executing Task...
    ├── 🤖 Agent: Orquestrador de Testes
    │       Status: In Progress
    ├── 🤖 Agent: Levantador de Cenários de Testes
    │       Status: ✅ Completed
    ├── 🤖 Agent: Levantador de Cenários de Testes
    │       Status: ✅ Completed
    ├── 🤖 Agent: Modelador BDD
    │       Status: ✅ Completed
    ├── 🤖 Agent: Automatizador de Testes
    │       Status: ✅ Completed
    ├── 🤖 Agent: Executor de Testes
    │       Status: ✅ Completed
    └── 🤖 Agent: Orquestrador de Testes
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 8d49e503-66f4-45b4-bcf6-a1ca4d62f0dc
│      Assigned to: Levantador de Cenários de Testes
│      Status: ✅ Completed
│   └── 🤖 Agent: Levantador de Cenários de Testes
│           Status: ✅ Completed
├── 📋 Task: 54a2753e-3530-475d-8967-7c626a614a5a
│      Assigned to: Modelador BDD
│      Status: ✅ Completed
│   └── 🤖 Agent: Modelador BDD
│           Status: ✅ Completed
├── 📋 Task: 50c8e2b7-19ca-4031-80fd-5d232c23b125
│      Assigned to: Automatizador de Testes
│      Status: ✅ Completed
│   └── 🤖 Agent: Automatizador de Testes
│           Status: ✅ Completed
├── 📋 Task: f81fc6ea-0b18-44ef-90aa-01e18d327614
│      Assigned to: Executor de Testes
│      Status: ✅ Completed
│   └── 🤖 Agent: Executor de Testes
│           Status: ✅ Completed
└── 📋 Task: e65e6a47-9122-4f2c-9e7a-e1f5afffa6dc
       Assigned to: Orquestrador de Testes
       Status: ✅ Completed
    ├── 🤖 Agent: Orquestrador de Testes
    │       Status: In Progress
    ├── 🤖 Agent: Levantador de Cenários de Testes
    │       Status: ✅ Completed
    ├── 🤖 Agent: Levantador de Cenários de Testes
    │       Status: ✅ Completed
    ├── 🤖 Agent: Modelador BDD
    │       Status: ✅ Completed
    ├── 🤖 Agent: Automatizador de Testes
    │       Status: ✅ Completed
    ├── 🤖 Agent: Executor de Testes
    │       Status: ✅ Completed
    └── 🤖 Agent: Orquestrador de Testes
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e65e6a47-9122-4f2c-9e7a-e1f5afffa6dc                                                                     │
│  Agent: Orquestrador de Testes                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8e79a79a-aaed-415a-9fcc-9bee306002a5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯